In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from random import randint
import re
from bs4 import BeautifulSoup
import pandas as pd
from collections import defaultdict
from time import sleep

# Scrape DMV website

**Objective: ** Find out which office in CA has the soonest 'first available appointment date' for getting a new ID card/Driver's License. 

**Background: ** Sometimes, it's better to drive an hour away to wait in a shorter line at the DMV.

**Set-backs: **Website uses reCAPTCHA to prove you are human when submitting the form.

In [ ]:
### Fill out your personal information after the colons.
personal_information = {
    'firstname': """input firstname here""",
    'lastname': """input last name here""",
    'phonenumber': """input phone number separated by dashes here. ie: 123-456-7890"""
}

In [4]:
#dictionary to store the DMV office name and their first available appointment date. 
dict_office = defaultdict()

In [2]:
safaridriver = webdriver.Safari()
safaridriver.maximize_window()

#open base url
url = 'https://www.dmv.ca.gov/wasapp/foa/startOfficeVisit.do'
safaridriver.get(url)

In [3]:
#get list of all CA dmv offices
html = safaridriver.find_element_by_tag_name('html').get_attribute('innerHTML')
soup = BeautifulSoup(html,'html.parser')
offices = list(filter(None, [i.text for i in soup.find_all('option')]))

In [10]:
def fill_dropdown(selection):
    """Select item from a dropdown menu"""
    drop_down=Select(safaridriver.find_element_by_xpath('//*[@id="officeId"]'))
    drop_down.select_by_visible_text(selection)
    sleep(randint(3,6))
    
def click_button(xpath_to_button):
    """Click on a button in the form."""
    button=safaridriver.find_element_by_xpath(xpath_to_button).click()  
    sleep(randint(3,6))
    
def fill_textbox(xpath_to_box, text):
    """Input any text into a textbox"""
    name1=safaridriver.find_element_by_xpath(xpath_to_box)
    name1.send_keys(text)
    sleep(randint(3,6))
    name1.send_keys(Keys.ENTER)
    sleep(randint(3,6))

def fill_out_form(office, personal_information):
    """Fill out all required information on DMV website"""
    #select office from drop-down menu
    fill_dropdown(office)

    #select how many items I would like to process: 1 item
    click_button('//*[@id="one_task"]')  
    
    #select reason for visit: Apply for, replace, or renew a California DL/ID
    click_button('//*[@id="taskCID"]')  

    #fill textboxes with personal information
    fill_textbox('//*[@id="first_name"]', personal_information['firstname'])
    fill_textbox('//*[@id="last_name"]', personal_information['lastname'])
    fill_textbox('//*[@id="areaCode"]', personal_information['phonenumber'].split('-')[0])
    fill_textbox('//*[@id="telPrefix"]', personal_information['phonenumber'].split('-')[1])
    fill_textbox('//*[@id="telSuffix"]', personal_information['phonenumber'].split('-')[2])

def get_date():
    """Find first available appointment for this office"""
    html = safaridriver.find_element_by_tag_name('html').get_attribute('innerHTML')
    soup = BeautifulSoup(html,'html.parser')
    date = soup.find(text=re.compile('The first available appointment for this office is on:'))\
    .next.next.find('strong').text.replace('\n','').replace('\t','').strip()
    dict_office[office] = date
    sleep(randint(3,6))

In [ ]:
for office in offices:
    if office == offices[0]:
        fill_out_form(office)
    try:
        #click: Continue
        click_button('//*[@id="findOffice"]/fieldset/div[8]/input[2]')
    except NoSuchElementException:

        #Sometimes the "Continue" button doesn't exist. It reappears when you "Clear Fields" and refill form. 
        clear_button = safaridriver.find_element_by_link_text('Clear Fields').click()
        fill_out_form(office)

        #click: Continue
        click_button('//*[@id="findOffice"]/fieldset/div[8]/input[2]')


        """~~~~~Code to solve reCAPTCHA should go here~~~~~"""
        """~~~~~Code to solve reCAPTCHA should go here~~~~~"""
        """~~~~~Code to solve reCAPTCHA should go here~~~~~"""


        #Get date of first available appointment and save result into dictionary.
        get_date()

        #Go back to select a different office. The form will already be filled out. 
        back_button = safaridriver.find_element_by_link_text('Back').click()